In [ ]:
import duckdb
import os

if (ENV := os.getenv("ENV")) is None:
    raise RuntimeError("ENV must be set")
if (DUCKDB_PATH := os.getenv("DUCKDB_PATH")) is None:
    raise RuntimeError("DUCKDB_PATH must be set")
if (CHROMA_DB_DIR := os.getenv("CHROMA_DB_DIR")) is None:
    raise RuntimeError("CHROMA_DB_PATH must be set")
if (GEMINI_API_KEY := os.getenv("GEMINI_API_KEY")) is None:
    raise RuntimeError("GEMINI_API_KEY must be set")

In [ ]:
from typing import Optional
from datetime import date

from pydantic import BaseModel

class RagRecord(BaseModel):
    doc_id: int
    document_title: str
    last_modified: date
    document_text: str
    ticket_id: Optional[str]
    priority: Optional[str]
    department: Optional[str]
    resolution_summary: Optional[str]
    rag_chunk: str

    def to_metadata(self) -> "ChunkMetadata":
        return ChunkMetadata(
            **self.model_dump(include=ChunkMetadata.model_fields.keys())
        )


class ChunkMetadata(BaseModel):
    doc_id: int
    document_title: str
    last_modified: date
    ticket_id: Optional[str]
    priority: Optional[str]
    department: Optional[str]
    resolution_summary: Optional[str]


In [ ]:
import google.generativeai as genai


genai.configure(api_key=GEMINI_API_KEY)


def stream_rag_records(db_path: str, batch_size: int=10) -> list[RagRecord]:
    """
    Stream rows from the docs_for_rag table in DuckDB in fixed-size batches.

    The query cursor fetches up to `batch_size` rows at a time using `fetchmany`,
    which avoids loading the entire table into memory. Each batch is converted
    into a list of RagRecord objects and yielded to the caller.

    The `while True` loop terminates naturally when DuckDB returns an empty list,
    signalling that all rows have been consumed.
    """

    sql = f"SELECT * FROM {ENV}_marts.docs_for_rag"
    con = duckdb.connect(db_path, read_only=True)
    result = con.execute(sql)
    col_names = [c[0] for c in result.description]

    while True:
        rows = result.fetchmany(batch_size)
        if not rows:
            break

        yield [RagRecord(**dict(zip(col_names, row))) for row in rows]

    con.close()


def embed_texts(texts: list[str]) -> list[list[float]]:
    """Embed a batch of strings using Gemini embeddings."""
    
    response = genai.embed_content(
        model="gemini-embedding-001",
        content=texts
    )
    
    return response['embedding']


def chunk_text(text: str, max_words=500) -> list[str]:
    """
    Simple word-based chunker. Splits text into chunks of up to `max_words`
    while preserving word boundaries. Suitable for small datasets and demo RAG.
    """
    words = text.split()
    if not words:
        return []

    chunks = []
    for i in range(0, len(words), max_words):
        chunk_words = words[i : i + max_words]
        chunks.append(" ".join(chunk_words))

    return chunks
    

    
        

In [ ]:
import chromadb


client = chromadb.PersistentClient(path=os.getenv("CHROMA_DB_DIR"))


collection = client.get_or_create_collection(
    name="policy_rag",
    metadata={"hnsw:space": "cosine"} 
)


for records in stream_rag_records(DUCKDB_PATH):
    all_chunks = []
    all_ids = []
    all_metadatas = []

    for rec in records:
        chunks = chunk_text(rec.rag_chunk)

        for i, chunk in enumerate(chunks):
            meta = rec.to_metadata().model_dump(mode="json")
            meta["num_chunks"] = len(chunks)
            meta["chunk_index"] = i
            all_chunks.append(chunk)
            all_ids.append(f"{rec.doc_id}_{rec.ticket_id}_{i}")
            all_metadatas.append(meta)
            
    embeddings = embed_texts(all_chunks) 
    
    collection.add(
                ids=all_ids,
                embeddings=embeddings,
                documents=all_chunks,
                metadatas=all_metadatas
    )

    

In [ ]:
# sanity check that number of records pulled from database equals number of entries in chroma.
# We'd expect this not to match if there were larger chunks, but in the provided demo data it should be the same

con = duckdb.connect(DUCKDB_PATH, read_only=True)
assert len(con.query(f"SELECT * FROM {ENV}_marts.docs_for_rag").fetchall()) == collection.count()

In [ ]:
# Test metadata parsing/upload: Roll up data in mart and collection to check they are the same

from collections import Counter

import duckdb
import pandas as pd


def duckdb_rollup(db_path="/app/database/mydb.duckdb"):
    con = duckdb.connect(db_path, read_only=True)
    df = con.execute(
        f"""
        SELECT department, COUNT(*) AS duckdb_count
        FROM {ENV}_marts.docs_for_rag
        GROUP BY department
        ORDER BY duckdb_count DESC
        """
    ).df()
    con.close()
    return df


def chroma_rollup(collection, batch_size=500):
    dept_counter = Counter()
    offset = 0

    while True:
        res = collection.get(
            include=["metadatas"],
            limit=batch_size,
            offset=offset
        )

        metadatas = res.get("metadatas", [])
        if not metadatas:
            break

        for m in metadatas:
            dept = m.get("department")
            if dept is not None:
                dept_counter[dept] += 1

        offset += batch_size

    df = pd.DataFrame(
        [{"department": d, "chroma_count": c} for d, c in dept_counter.items()]
    ).sort_values("chroma_count", ascending=False)
    return df

from IPython.display import display, HTML


duckdb_df = duckdb_rollup()
chroma_df = chroma_rollup(collection)


html = f"""
<div style="display:flex; gap:40px;">
    <div>{duckdb_df.to_html(index=False)}</div>
    <div>{chroma_df.to_html(index=False)}</div>
</div>
"""

display(HTML(html))

In [ ]:
# Search function to test semantic search

def search(query: str, top_k: int = 1, where: dict | None = None) -> list[dict]:
    embedding = embed_texts([query])[0]

    
    result = collection.query(
        query_embeddings=[embedding],
        n_results=top_k,
        where=where
    )

    return {
        "documents": result["documents"][0],
        "metadatas": result["metadatas"][0],
        "distances": result["distances"][0],
        "ids": result["ids"][0],
    }

In [ ]:
# Investigate unique values for filtering

con.query(f"select distinct department from {ENV}_marts.docs_for_rag").fetchall()

In [ ]:
search("hotel policy", 2)

In [ ]:
search("how much monthly paid time off do employees get", 1, {"department": "HR"})